In [1]:
import wandb

# This is importent when we want to call this as a python script, because jupyter naturally has a higher recursion depth
from icecream import ic
import os
import sys
sys.setrecursionlimit(3000)

# Print the PID when using nohup
ic(os.getpid())

ic| os.getpid(): 4083743


4083743

In [2]:
from datetime import datetime
import pickle
from typing import Optional
from gymportal.data.ev_generators import SklearnGenerator, get_data, extract_training_data
from gymportal.auxilliaries.file_utils import get_persistent_folder
import pytz
from sklearn.mixture import BayesianGaussianMixture
import numpy as np


class ScalableSklearnGenerator(SklearnGenerator):

    def __init__(
        self,
        period,
        battery_generator,
        model,
        scaler,
        frequencies_per_hour,
        duration_multiplicator=1,
        arrival_min=0,
        arrival_max=24,
        duration_min=0.0833,
        duration_max=48,
        energy_min=0.5,
        energy_max=150,
        seed=None
    ):
        super().__init__(
            period,
            battery_generator,
            model,
            frequencies_per_hour,
            duration_multiplicator,
            arrival_min, arrival_max,
            duration_min, duration_max,
            energy_min,
            energy_max,
            seed
        )

        self.scaler = scaler

    def _sample(self, n_samples: int):
        """ Generate random samples from the fitted model.

        Args:
            n_samples (int): Number of samples to generate.

        Returns:
            np.ndarray: shape (n_samples, 3), randomly generated samples. Column 1 is
                the arrival time in hours since midnight, column 2 is the session duration in hours,
                and column 3 is the energy demand in kWh.
        """
        if n_samples > 0:
            ev_matrix, _ = self.sklearn_model.sample(n_samples)
            ev_matrix = self.scaler.inverse_transform(ev_matrix)
            return self._clip_samples(ev_matrix)
        else:
            return np.array([])


def get_generator(site, model_path: str, battery_generator, token: Optional[str] = None, seed: Optional[int] = None,
                           frequency_multiplicator=10, duration_multiplicator=1):
    """

    Args:
        site: The site which is used as a data source for the generative model.
        battery_generator: The generator for EV batteries.
        token: The token to access acn-data.
        seed: A seed for random number generator
        frequency_multiplicator: A multiplicator for the arrival frequencies of EVs, e.g., a higher value makes it
            more likely for an EV to arrive at a given point in time.

    Returns:

    """
    timezone = pytz.timezone('America/Los_Angeles')
    data = get_data(
        site,
        token,
        drop_columns=(),
        start=datetime(2018, 3, 25, tzinfo=timezone),
        end=datetime(2020, 5, 31, tzinfo=timezone)
    )
    X = extract_training_data(data)

    try:
        with open(model_path, "rb") as f:
            gmm, scaler = pickle.load(f)
    except FileNotFoundError:
        print(f"No existing GMM found for site={site}!")

    connection_time = X[:, 0]

    frequencies, _ = np.histogram(connection_time, bins=range(0, 25, 1))
    frequencies = np.array(frequencies) / np.sum(frequencies)

    generator = ScalableSklearnGenerator(
        period=1,
        model=gmm,
        scaler=scaler,
        frequencies_per_hour=frequencies * frequency_multiplicator,
        battery_generator=battery_generator,
        duration_multiplicator=duration_multiplicator,
        seed=seed
    )

    return generator

E0000 00:00:1742384481.164589 4083743 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742384481.167263 4083743 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# from gymportal.data.ev_generators import get_standard_generator, RealWorldGenerator
from acnportal.acnsim import Linear2StageBattery
from gymportal.data.battery_generators import CustomizableBatteryGenerator
from gymportal.sim import get_charging_network, Recomputer, EvaluationSimulator, SimGenerator
from datetime import datetime, timedelta

import pytz
timezone = pytz.timezone("America/Los_Angeles")


# charging_network = get_charging_network('simple_acn', basic_evse=True, voltage=208,
#                                         network_kwargs={
#                                             'station_ids': ['CA-504', 'CA-503', 'CA-502', 'CA-501'],
#                                             # 'station_ids': ['CA-501'],
#                                             "aggregate_cap": 32 * 208 / 1000})

charging_network = get_charging_network('caltech', basic_evse=True, voltage=208,
                                        network_kwargs={"transformer_cap": 150})

battery_generator = CustomizableBatteryGenerator(
    voltage=208,
    period=1,
    battery_types=[
        Linear2StageBattery],
    max_power_function="normal",
)

# ev_generator = RealWorldGenerator(battery_generator=battery_generator, site='caltech', period=1)
ev_generator = get_generator(
    'caltech',
    "triple_gmm+sc.pkl",
    battery_generator,
    seed=42,
    frequency_multiplicator=10,
    duration_multiplicator=2
)


# TODO Use time intervals and GMMs from https://github.com/chrisyeh96/sustaingym/blob/main/sustaingym/envs/evcharging/utils.py
# I.e., train on generated data, evaluate on new generated data and real data from the same interval
# optional: compare to "out-of-distribution" data from different interval

train_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=1,
    n_intervals=46 * 7,
    start_date=timezone.localize(datetime(2019, 1, 1)),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(train_generator.end_date + timedelta(days=1))

eval_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=7 * 4,
    n_intervals=1,
    start_date=train_generator.end_date + timedelta(days=1),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(eval_generator.end_date + timedelta(days=1))

# validation_generator = SimGenerator(
#     charging_network=charging_network,
#     simulation_days=14,
#     n_intervals=1,
#     start_date=eval_generator.end_date + timedelta(days=1),
#     ev_generator=ev_generator,
#     recomputer=Recomputer(recompute_interval=10, sparse=True),
#     sim_class=EvaluationSimulator,
# )

# ic(validation_generator.end_date + timedelta(days=1))
pass

ic| train_generator.end_date + timedelta(days=1): datetime.datetime(2019, 11, 20, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| eval_generator.end_date + timedelta(days=1): datetime.datetime(2019, 12, 19, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)


In [4]:
from src.pv.pv import read_pv_data

df_pv = read_pv_data("../pv_150kW.csv")
df_pv.describe()

,P,G(i),H_sun,T2m,WS10m,Int
count,140256.000000,140256.000000,140256.000000,140256.000000,140256.000000,140256.0
mean,30395.798319,267.983145,16.748906,17.931872,1.654366,0.0
std,40648.543625,356.157287,22.166668,7.394042,0.867817,0.0
min,0.000000,0.000000,-5.880000,-2.230000,0.000000,0.0
25%,0.000000,0.000000,0.000000,12.360000,0.970000,0.0
50%,87.000000,7.260000,0.520000,17.310000,1.590000,0.0
75%,65764.500000,564.280000,31.982500,23.010000,2.280000,0.0
max,132601.500000,1143.220000,78.090000,46.460000,7.930000,0.0


In [5]:
from gymportal.environment import *
from src.observations import minute_observation_stay
from src.pv.observations import pv_observation_mean
from src.pv.rewards import *
from src.pv.metrics import *
from gymportal.evaluation import *


observation_objects = [
    charging_rates_observation_normalized(),
    percentage_of_magnitude_observation(),
    diff_pilots_charging_rates_observation_normalized(),
    cyclical_minute_observation(),
    cyclical_day_observation(),
    cyclical_month_observation(),
    minute_observation_stay(),
    energy_delivered_observation_normalized(),
    num_active_stations_observation_normalized(),
    pilot_signals_observation_normalized(),
    pv_observation_mean(df_pv),
]

reward_objects = [
    pv_utilization_reward(df_pv),
    # grid_use_penalty(df_pv),
    unused_pv_penalty(df_pv),
    charging_reward(),
    # soft_charging_reward_pv_weighted(df_pv, transformer_cap=150),
]

metrics = {
    "SoC >= 90%": percentage_soc,
    "mean SoC": mean_soc,
    "median SoC": median_soc,
    "prop feasible steps": proportion_of_feasible_charging,
    "prop feasible charge": proportion_of_feasible_charge,
    "pv utilization": lambda sim: pv_utilization_metric(sim, df_pv),
    "grid usage": lambda sim: grid_use_metric(sim, df_pv),
    "unused pv": lambda sim: unused_pv_metric(sim, df_pv),
}

In [6]:
# import dill as pickle

# with open("../caltech_#stations=54_#days=7_#intervals=46_seed=8734956.pkl", "rb") as file:
#     train_generator = pickle.load(file)

In [ ]:
train_generator.seed = 8734956 
_ = train_generator.reset()

iter = 0

while train_generator._current_date != train_generator.start_date:
    _ = train_generator.next()

    ic(iter)
    ic(train_generator._current_date)
    iter += 1

ic| iter: 0
ic| train_generator._current_date: datetime.datetime(2019, 1, 3, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| iter: 1
ic| train_generator._current_date: datetime.datetime(2019, 1, 4, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| iter: 2
ic| train_generator._current_date: datetime.datetime(2019, 1, 5, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| iter: 3
ic| train_generator._current_date: datetime.datetime(2019, 1, 6, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| iter: 4
ic| train_generator._current_date: datetime.datetime(2019, 1, 7, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| iter: 5
ic| train_generator._current_date: datetime.datetime(2019, 1, 8, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| iter: 6
ic| train_generator._current_date: datetime.datetime(2019, 1, 9, 0, 0, tzinfo=<DstTzInfo 'Amer

In [8]:
steps_per_epoch = 0
for eval_sim in train_generator._sim_memory:
    steps_per_epoch += len(eval_sim.event_queue.queue)

ic(steps_per_epoch)

ic| steps_per_epoch: 59342


59342

In [9]:
eval_generator.seed = 8734956
_ = eval_generator.reset()

iter = 0

while eval_generator._current_date != eval_generator.start_date:
    _ = eval_generator.next()

    ic(iter)
    ic(eval_generator._current_date)
    iter += 1
    
steps_per_epoch_eval = 0
for eval_sim in eval_generator._sim_memory:
    steps_per_epoch_eval += len(eval_sim.event_queue.queue)

ic(steps_per_epoch_eval)

ic| iter: 0
ic| eval_generator._current_date: datetime.datetime(2019, 11, 20, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| steps_per_epoch_eval: 5149


5149

In [10]:
train_config = {"observation_objects": observation_objects, "action_object": zero_centered_single_charging_schedule_normalized(),
                "reward_objects": reward_objects,
                "simgenerator": train_generator,
                "meet_constraints": True}

eval_config = train_config | {'simgenerator': eval_generator}
# validation_config = train_config | {'simgenerator': validation_generator}

In [11]:
from acnportal.algorithms import UncontrolledCharging, SortedSchedulingAlgo, last_come_first_served, \
    first_come_first_served

model = ACNSchedule(SortedSchedulingAlgo(first_come_first_served))

In [12]:
from tqdm import tqdm
import pandas as pd

def create_df(model, name: str, env, steps_per_epoch):
    df_list = []
    
    done = False
    old_obs, _ = env.reset()

    for _ in tqdm(range(steps_per_epoch)):

        iface = env.unwrapped.interface
        action = model.get_action(old_obs, iface)

        new_obs, rew, terminated, truncated, _ = env.step(
            action)
        done = terminated or truncated

        df_list.append([old_obs.tolist(), action.tolist(), rew, done])

        if done:
            new_obs, _ = env.reset()
            done = False
            
        old_obs = new_obs
        
    df = pd.DataFrame(df_list, columns=['observation', 'action', 'reward', 'done'])
    df.to_parquet(f'{name}.parquet.gzip', compression='gzip')
    

In [13]:
from src.cleanRL.environment import make_env

create_df(model, "FCFS_gen_triple_46_weeks_training", make_env(train_config, 0.99, 0)(), steps_per_epoch)
create_df(model, "FCFS_gen_triple_46_weeks_validation", make_env(eval_config, 0.99, 0)(), steps_per_epoch_eval)

  0%|          | 231/59342 [00:21<1:01:15, 16.08it/s]/home/guest/tsturm/acn-experiments/.conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/guest/tsturm/acn-experiments/.conda/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5149/5149 [08:03<00:00, 10.65it/s]
